***Step 1***

In [1]:
from PyPDF2 import PdfReader, PdfWriter

input_path = "combined 15.pdf"
output_path = "SDM_trimmed_output.pdf"

reader = PdfReader(input_path)
writer = PdfWriter()

total_pages = len(reader.pages)

for i in range(0, total_pages, 6):
    writer.add_page(reader.pages[i])

#Save
with open(output_path, "wb") as f:
    writer.write(f)

print(f"Trimmed saved as: {output_path}")

Trimmed saved as: SDM_trimmed_output.pdf


***Step 2***

In [2]:
#ROTATED PDF
from PyPDF2 import PdfReader, PdfWriter
input_pdf = "combined 15.pdf"
rotated_pdf = "SDM_rotated_trimmed_output.pdf"
reader = PdfReader(input_pdf)
writer = PdfWriter()

for page in reader.pages:
    page.rotate(90)
    writer.add_page(page)

with open(rotated_pdf,"wb") as f:
    writer.write(f)

print("PDF rotated and saved: ", rotated_pdf)


PDF rotated and saved:  SDM_rotated_trimmed_output.pdf


***Step 3***

In [3]:
import pdfplumber
import re
import pandas as pd

pdf_path = "SDM_rotated_trimmed_output.pdf"
extracted_data = []

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        print(f"\n--- page {i+1} ---")
        text = page.extract_text()
        if  text:
            
            lines = text.strip().split('\n')
            for idx, line in enumerate(lines):
                print(f" {idx+1}: {line}")
        else:
            print(f"No text extracted")
        


--- page 1 ---
 1: (cid:1)(cid:4)(cid:1)(cid:3)(cid:2)(cid:4)(cid:3)(cid:3)(cid:1)(cid:3)(cid:4)(cid:2)(cid:2)(cid:1)(cid:3)(cid:3)(cid:2)(cid:2)(cid:4)(cid:1)(cid:3)(cid:4)(cid:2)(cid:3)(cid:2)(cid:3)(cid:1)(cid:4)(cid:3)(cid:1)(cid:2)(cid:4)(cid:4)(cid:4)(cid:1)(cid:3)(cid:2)(cid:2)(cid:3)(cid:1)(cid:2)(cid:4)(cid:4)(cid:1)(cid:3)(cid:4)(cid:1)(cid:2)(cid:4)(cid:3)(cid:3)(cid:2)(cid:1)(cid:4)(cid:4)(cid:1)(cid:3)(cid:4)(cid:3)(cid:1)(cid:3)(cid:2)(cid:1)(cid:1)(cid:3)
 2: 3025/1/9/2 ********AUTO**ALL FOR AADC 303
 3: 97034745 TG3K01
 4: DONNA WORLEY
 5: 2691 BILL CARRUTH PKWY
 6: HIRAM GA 30141-3760

--- page 2 ---
 1: (cid:3)(cid:3)(cid:2)(cid:2)(cid:1)(cid:1)(cid:2)(cid:3)(cid:2)(cid:2)(cid:3)(cid:4)(cid:4)(cid:4)(cid:4)(cid:3)(cid:4)(cid:1)(cid:2)(cid:2)(cid:4)(cid:1)(cid:1)(cid:3)(cid:1)(cid:2)(cid:3)(cid:2)(cid:4)(cid:1)(cid:3)(cid:2)(cid:1)(cid:3)(cid:4)(cid:4)(cid:1)(cid:1)(cid:2)(cid:1)(cid:2)(cid:4)(cid:4)(cid:1)(cid:2)(cid:1)(cid:3)(cid:2)(cid:3)(cid:4)(cid:3)(cid:4)(cid:3

***Step 4***

In [4]:
import pdfplumber
import pandas as pd
import re

pdf_path = "SDM_rotated_trimmed_output.pdf"
extracted_data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue
        lines = text.strip().split('\n')

        record = {"full_name": "", "household_id": "", "full_address": "", "creative_version": ""}

        for i, line in enumerate(lines):
            line = line.strip()

            #
            if re.match(r"^\d{7,} +[A-Z0-9]+$", line):
                parts = line.split()
                if len(parts) >= 2:
                    record["household_id"] = parts[0]
                    record["creative_version"] = parts[1]

            # M
            elif re.match(r"^[A-Z][A-Z\s\-']+$", line) and not record["full_name"]:
                record["full_name"] = line

            # Match addre
            elif re.match(r"^\d+ .+", line) and not record["full_address"]:
                address_line = line
                if i + 1 < len(lines):
                    city_state_zip = lines[i + 1].strip()
                    if re.search(r"\b[A-Z]{2} \d{5}(-\d{4})?\b", city_state_zip):
                        record["full_address"] = f"{address_line}, {city_state_zip}"

            # Sav
            if all(record.values()):
                extracted_data.append(record.copy())
                record = {"full_name": "", "household_id": "", "full_address": "", "creative_version": ""}

# Create DataFrame
df = pd.DataFrame(extracted_data)
df

df.to_excel("SDM_output_From_PDF.xlsx", index=True)




***Step 5***